In [12]:
from googleapiclient.discovery import build
import json, time, sys
from collections import deque
import pandas as pd

video_collected = []

def formatComment(comment):
    doc = {}
    doc['id']=comment['id']
    doc['comment'] = comment['snippet']['topLevelComment']['snippet']['textOriginal']
    doc['user'] = comment['snippet']['topLevelComment']['snippet']['authorDisplayName']
    try:
        doc['userId'] = comment['snippet']['topLevelComment']['snippet']['authorChannelId']['value']
    except:
        doc['userId'] = ""
    doc['likeCount'] = comment['snippet']['topLevelComment']['snippet']['likeCount']
    doc['publishedAt']=comment['snippet']['topLevelComment']['snippet']['publishedAt']
    doc['totalReplyCount'] = comment['snippet']['totalReplyCount']
    doc['videoId'] = comment['snippet']['videoId']
    doc['parentId'] = ""
    return doc

def formatReply(reply):
    doc = {}
    doc['id']=reply['id']
    doc['comment'] = reply['snippet']['textOriginal']
    doc['user'] = reply['snippet']['authorDisplayName']
    try:
        doc['userId'] = reply['snippet']['authorChannelId']['value']
    except:
        doc['userId'] = ""
    doc['likeCount'] = reply['snippet']['likeCount']
    doc['publishedAt']=reply['snippet']['publishedAt']
    doc['totalReplyCount'] = 0
    doc['videoId'] = reply['snippet']['videoId']
    doc['parentId'] = reply['snippet']['parentId']
    return doc

class Collector:
    def __init__(self,collected_videos=set()):
        with open('config.json') as config:
            config=json.load(config)
            keys = config['api_keys']
        self.youtubes = [build('youtube', 'v3', developerKey=key) for key in keys]
        self.youtube = self.youtubes[0]
        self.current_queries = 0
        self.collected_videos = collected_videos
        
        
    def roll(self):
        try:
            self.youtube = self.youtubes[self.current_queries//9000]
        except:
            print('Quota exceeded,sleeping')
            time.sleep(60*60*24)
            self.youtube=self.youtubes[0]
            self.current_queries = 0
    
    #function to get the top videos for each category according to a sorting method
    def getVideos(self, categories,method='relevance'):
        videos_metadata=[]
        part=["id","snippet"]
        for category in categories:
            videos = []

            while True:
                try:
                    resp = self.youtube.search().list(regionCode='us',part='snippet',maxResults=50,type='video'
                                                    ,order=method,relevanceLanguage='en',videoCategoryId=category)
                    result = resp.execute()
                    self.current_queries+=100
                    self.roll()
                    break
                except:#service unavailable
                    time.sleep(60)
            for item in result['items']:
                if item['id']['videoId'] in self.collected_videos:
                    continue
                formattedResult = {}
                formattedResult['videoId'] = item['id']['videoId']
                formattedResult['publishedAt']=item['snippet']['publishedAt']
                formattedResult['title']=item['snippet']['title']
                formattedResult['description']=item['snippet']['description']
                formattedResult['channelTitle']=item['snippet']['channelTitle']
                formattedResult['channelId']=item['snippet']['channelId']
                formattedResult['liveBroadCastContent'] = (item['snippet']['liveBroadcastContent']!='none')
                formattedResult['category'] = category
                formattedResult['method'] = method
                videos.append(formattedResult)
                self.collected_videos.add(item['videoId'])
            while result:
                if (len(videos) >= 1000) or ('nextPageToken' not in result):
                    break
                while True:
                    try:
                        resp = self.youtube.search().list(regionCode='us',pageToken=result['nextPageToken'],part='snippet',maxResults=50,type='video'
                                                    ,order=method,relevanceLanguage='en',videoCategoryId=category)
                        result = resp.execute()
                        self.current_queries+=100
                        self.roll()
                        break
                    except:
                        time.sleep(60)
                for item in result['items']:
                    if item['id']['videoId'] in self.collected_videos:
                        continue
                    formattedResult = {}
                    formattedResult['videoId'] = item['id']['videoId']
                    formattedResult['publishedAt']=item['snippet']['publishedAt']
                    formattedResult['title']=item['snippet']['title']
                    formattedResult['description']=item['snippet']['description']
                    formattedResult['channelTitle']=item['snippet']['channelTitle']
                    formattedResult['channelId']=item['snippet']['channelId']
                    formattedResult['liveBroadCastContent'] = (item['snippet']['liveBroadcastContent']!='none')
                    formattedResult['category'] = category
                    formattedResult['method'] = method 
                    videos.append(formattedResult)
                    self.collected_videos.add(item['videoId'])
            videos_metadata.extend(videos)
        return videos_metadata
    
    #function to get video metadata and view/like/comment counts
    def getVideoStatistics(self,videos):
        for video in videos:
            print(video['videoId'])
            while True:
                try:
                    resp = self.youtube.videos().list(part=['snippet','contentDetails','topicDetails','statistics'],id=video['videoId'])
                    result = resp.execute()
                    self.current_queries+=1
                    self.roll()
                    break
                except Exception as e:
                    print(e)
                    time.sleep(60)
            for item in result['items']:
                try:
                    video['tags'] = item['snippet']['tags']
                except:
                    video['tags'] = []
                try:
                    video['language'] = item['snippet']['defaultLanguage']
                except:
                    video['language'] = ''
                video['duration'] = item['contentDetails']['duration']
                try:
                    video['topics'] = item['topicDetails']['topicCategories']
                except:
                    video['topics'] = []
                video.update(item['statistics'])
                video['method'] = 'from_channel'
                try:
                    video['category'] = item['snippet']['categoryId']
                except:
                    video['category'] = ''
        return videos
    
    #function to get channel metadata
    def getChannelStatistics(self,channels):
        channels_metadata=[]
        for channelId in channels:
            while True:
                try:
                    resp= self.youtube.channels().list(part=['id','statistics','snippet'],id=channelId)
                    result = resp.execute()
                    self.current_queries+=1
                    self.roll()
                    break
                except Exception as e:
                    raise(e)
                    time.sleep(60)
            if 'items' not in result:
                continue
            channel_data = {}
            channel_data['title']=result['items'][0]['snippet']['title']
            channel_data['id'] = result['items'][0]['id']
            channel_data['viewCount'] = result['items'][0]['statistics']['viewCount']
            try:
                channel_data['subscriberCount'] = result['items'][0]['statistics']['subscriberCount']
            except:
                channel_data['subscriberCount'] = None
            try:
                channel_data['description'] = result['items'][0]['snippet']['description']
            except:
                channel_data['description'] = ''
            try:
                channel_data['publishedAt'] = result['items'][0]['snippet']['publishedAt']
            except:
                channel_data['publishedAt'] = None
            channel_data['videoCount'] = result['items'][0]['statistics']['videoCount']
            channels_metadata.append(channel_data)
        return channels_metadata

    #function to backup currently collected videos
    def backup(self):
        with open('Data/temp_videos.jsonl','a',encoding='utf-8') as file:
            for video in self.videos_metadata:
                json.dump(video,file,ensure_ascii=False)
                file.write('\n')
        with open('Data/backup.txt','w') as backup:
            for pageToken,channel in self.page_queue:
                backup.write(str(pageToken)+','+str(channel))
                backup.write('\n')
        self.videos_metadata = []
        self.n_iter=0
        
    #gets top 50 videos from a channel
    def getFromChannel(self,channels,iter_to_backup=10):
        self.videos_metadata=[]
        part=["id","snippet"]
        self.n_iter=0
        for channel in channels:
            self.n_iter+=1
            if self.n_iter%iter_to_backup == 0:
                print(f'backing up data at {self.n_iter} iterations')
                self.backup()
            videos = []
            while True:
                try:
                    resp = self.youtube.search().list(regionCode='us',part='snippet',maxResults=50,type='video'
                                            ,order='viewCount',relevanceLanguage='en',channelId=channel['id'])
                    result = resp.execute()
                    self.current_queries+=100
                    self.roll()
                    break
                except Exception as e:#service unavailable
                    print(e)
                    time.sleep(60)
            for item in result['items']:
                formattedResult = {}
                formattedResult['videoId'] = item['id']['videoId']
                formattedResult['publishedAt']=item['snippet']['publishedAt']
                formattedResult['title']=item['snippet']['title']
                formattedResult['description']=item['snippet']['description']
                formattedResult['channelTitle']=item['snippet']['channelTitle']
                formattedResult['channelId']=item['snippet']['channelId']
                formattedResult['liveBroadCastContent'] = (item['snippet']['liveBroadcastContent']!='none')
                formattedResult['category'] = item['snippet']['categoryId']
                formattedResult['method'] = 'from_channel'
                videos.append(formattedResult)
            if result and 'nextPageToken' in result:
                self.page_queue.append((result['nextPageToken'],channel['id']))
            self.videos_metadata.extend(videos)

        self.backup()
        
        return True
    
    #gets up to "limit" comments for a given video
    def getComments(self,videoId,limit=float('inf')):
        comments = []
        response = self.youtube.commentThreads().list(part=["id","snippet","replies"],videoId=videoId,maxResults=100).execute()
        self.current_queries+=1
        self.roll()
        while response:
            for item in response['items']:
                formattedResult = formatComment(item)
                comments.append(formattedResult)
                if len(comments)%10000 ==0:
                    print(f'{len(comments)} collected for {videoId}')
                if len(comments)>=limit:
                    return comments
                if formattedResult['totalReplyCount']>0:
                    try:
                        for reply in item['replies']['comments']:
                            reply = formatReply(reply)
                            comments.append(reply)
                    except:
                        pass
            if 'nextPageToken' in response:
                while True:
                    try:
                        response = self.youtube.commentThreads().list(part=["id","snippet","replies"],videoId = videoId,maxResults=100,pageToken=response['nextPageToken']).execute()
                        self.current_queries+=1
                        self.roll()
                        break
                    except:
                        time.sleep(60)
            else:
                break
        return comments

# Main collection loop - Part 1

In [ ]:
try:
    with open('Data/categories.json') as file:
        categories = json.load(file)
except:
    categories = youtube.videoCategories().list(part='snippet',regionCode='US')
    with open('Data/categories.json','w') as file:
        json.dump(categories,file)


category_ids = [category['id'] for category in categories]
print(f'Collecting data for {len(category_ids)} categories using method')

collector = Collector(collected_videos=set())
videos=collector.getVideos(category_ids,method='viewCount')
print(f'Finished category queries {len(videos)} found')
videos = collector.getVideoStatistics(videos)
print(f'Finished collecting statistics')
with open('Data/video_metadata.jsonl','a',encoding='utf-8') as file:
    for line in videos:
        json.dump(line,file,ensure_ascii=False)
        file.write('\n')

In [ ]:
for video in videos:
    video_collected.append(video['videoId'])
    try:
        comments=collector.getComments(video['videoId'])
    except Exception as e:
        if 'disabled comments' in str(e):
            print(f'Video {video["videoId"]} has disabled comments')
            continue
    print(f'Collected {len(comments)} comments for {video["videoId"]}')
    with open('Data/comments.jsonl','a',encoding='utf-8') as file:
        for comment in comments:
            json.dump(comment,file,ensure_ascii=False)
            file.write('\n')
print('Done')

In [ ]:
channels = set([video['channelId'] for video in videos])
print(f'Collecting data for {len(channels)} channels')
channel_statistics = collector.getChannelStatistics(channels)
with open('Data/channel_metadata.jsonl','w',encoding='utf-8') as file:
     for channel in channel_statistics:
        json.dump(channel,file,ensure_ascii=False)
        file.write('\n')

# Collection loop - part 2 
## Popular videos from collected channels

In [ ]:
channels=pd.read_csv('channels_relevant.csv').to_dict(orient='records')

In [ ]:
collector.getFromChannel(channels,iter_to_backup=1)

In [19]:
#keep only top 3 videos due to time_constraints
videos = pd.read_json('Data/temp_videos.jsonl',lines=True)
vid = pd.read_json('Data/video_metadata.jsonl',lines=True)
videos_to_collect = []
for c,df in videos.groupby('channelId'):
    df = df[~df['videoId'].isin(vid['videoId'].tolist())]
    df = df[:3]
    videos_to_collect.append(df.copy())
videos_to_collect = pd.concat(videos_to_collect).reset_index(drop=True)

In [20]:
collector = Collector(collected_videos=set())
collector.current_queries=10757
videos_to_collect = videos_to_collect.to_dict(orient='records')
video_metadata = collector.getVideoStatistics(videos_to_collect)

n4N4bbIK2ls
b5kmkAACtg0
PRM-NAj63g8
gmQE4qdb9fg
2ovhbT-Iulc
qNQ2kV1OTPU
1O6Qstncpnc
8pR0o2fGyHg
ph8LyNIT9sg
USn8H910yxM
Lx1i2bCXv34
i_1dT_2Bz2E
wMSyS-24o8Q
RK574AygVLw
LYZiXzExUbs
gEK7Vt5X7CU
p3g9dIA001s
ktsOV-E_MTo
qyoeQsBF0FA
NRLf5Iq1Xiw
nagJ88k_hDI
3VCxkMvKUnA
6ufwxkurKKg
MbUvTbD9q4A
70ALNyiOe6k
3yAJIvEg0wo
_olkwh6lQ_s
own-HFbRVzA
otD5ECXjfIY
aGsrTHB8JpA
uG8V9dRqSsw
X6PJEmEHIaY
Lg-Ctg6k_Ao
QjGUFmvolqI
s_w6-neLVos
yc3ihuMtlo0
l6C59srUSco
QODZ9f8BT_I
D8WGhb0vh3I
TTGczb8uLA0
CFZWI8ExfuY
id00R-3OmJ0
A1JH9URsPT8
me8qgPNiZ-E
oimbQ1XuL-w
0EdjBxj_QKk
MtTfU5s0H8Q
hAfYCjQ4U4A
CWSX34OV8RY
aWfvxdT9okw
ZJVk1cE6O5M
ypXtUvPRJwM
4NxHONRhG1c
63H9CdTKSsw
2zAB1UlbYJw
pozYwTcPNZQ
wzySUQsFzAA
SDxYxkCMpyc
ZGJXWMkNcyQ
bnjbiocltF8
0o1TajSJvN8
I8lO1tGuhCM
R-yA4pfQ8a8
Pz9qRd-D9RE
OSHhB-RBDOg
lEB6Y9lxu7g
tx5qG67fS3g
J-nK0fZV7-8
WhLegZqTFu4
UzO9AfB2oSc
V24ONWLpVmY
HwbutTQ8Yow
9MZticL8vVs
2jPc392W7xs
tm9PRuEUZl4
LCJ-eAt79N0
dc3N_xqUKKA
CdrMVixNmOg
3Eo_GvDGP-g
bNdeQC8cUXo
5gVZBFv5U1c
FGghVNg1qBQ
X5z7ZjzUs1g
CqWf

dKO2tNYrHMQ
TP4Xtd6L7fw
e4-CuXW7oOc
2atGcP3bSxI
jltLSczolzM
_a8u4949t_k
MkCSGVecz_0
mdpFyyYzJHA
dGHzj1Pozl8
RXuQum713p4
UOxkGD8qRB4
fmI_Ndrxy14
4Q46xYqUwZQ
y1FtYNHVZC0
aLMmkiy6dNA
G-bLsrNHcpE
9svic7ldL2w
ksLwSIUljP4
h_vSEviFXIo
mcX-SQ2IS7M
MAlNm-1tyzI
8o32m_b4oqU
08fWWKhpxCg
g3gNXanV-GI
8sOpjk_9fD8
S1qQnUvETXM
lWBWV8WbMAY
Cr1FcQOe0gc
Cqhsh2iyopM
zadtG3D_yws
Hrfpp5DTWBA
Bni4cU24yZc
2zUDIgR9ra0
1y4vbJJX_ME
SDughwDukB0
pcCH_UolR7Y
7LjfuAQL2G0
qk4gZrBR9CU
bgPDW0ZpgJU
eGx4FWS7zE4
9ZgKOk1JD1Q
P9514y2WbbU
e0JHPTkMX7s
wh4flZKtLEc
HQHoKkXQbdM
MNTOX3hvCL8
x9NL4mscIHQ
pRf2JHw9LQg
fDunkFBT7RQ
FQciGRKxRqI
EClOIrJpX08
NnfaTux6AMI
G0mncEl4nVU
oKAQJEgH6MM
vMl-g-GJ1Ac
LQlWXuiAWc0
gZ2xSU836P8
uVcryekusbQ
N4a0EaUT9nE
kbkNb2fiLlM
UMpmoRlWD8Q
BoSXhLATGWE
tzMmCyFbp3w
8VBXlu-9XrM
SCVEoOr5qfQ
78kntDlbjMI
uUueoFbOCZM
iqerkzGqx0I
nK1jiAlatMY
SH7ELystIb0
CwQB-LHI3LU
3l_frT5T9JM
RLBHWIDQICo
kcDHNOx5zrw
Eg2y8fxT4dc
8zC0SmusUbM
o49XxK98AxQ
fZEnqFzw-_o
qH40OVMWgdU
2hlzGnJRYf8
EC8X2W-dDTw
Jm0ydkxLCFk
rEmHO1o6wZg
2zVp

CL0XCZlmwjU
zxsXRaGXnZY
S8BWttw_c2Q
jpA9cQ8Zxn4
GFmM7A_g_bA
hW-iF-tKGcs
WEjzF2Cx1UY
KAJsdgTPJpU
kvmasW_glUQ
Xa2PpYeGwt8
UKLfeb0K1KY
IZnThPxzRso
DhSz5BdtCrM
nhY1s6CZziE
W3tGqXyvJLM
qsqwn2ZvpL4
uTRZ4N1e6Wg
74_FdfYlaRo
DxoEHH2w2CU
vcCf90oMEVs
qM4cephTQuw
c7YcSrXjdNA
X_PU6t6IS38
Jal5s1HR3Do
UH1T5u9sWJk
RPk4tNfuXT0
0yA3Nuyde04
dyUTpuYte94
u19lZ0hNypY
T5za4-49nBs
jhRb-vRokB8
GmMRjS-D5FA
ZctlpzLEJvs
YTzTlBCDOS4
QoISYHup7hQ
TIw1OPH6QvM
pH0fhLBr-Mg
GwaRztMaoY0
By_JYrhx-WY
8-91y7BJ8QA
cvFaxhDUXy4
IinWIC0crxs
otOO5l9lmD0
OX0wrJuogLU
LMZdQkCnYgA
r-7FyO5sWEs
gyY5jvX7yf4
YiLhthFU_9g
HlpgseGq9Cg
TPA3FwrF_jA
-jWwk2dClEo
ZIMVePaQHq4
jHMintsw7HM
QTZCFTmSwO8
DSRVE6L8STs
ZLgsUDopO3c
vA4NS_S8h1s
3wsR6_MSZ_A
zhM3KsUUqzo
yp8Uk455BdI
DWgHqumUA8U
AMJ87ZQ6LDo
jFgazN_o6cs
pDHzS1Bshlw
JGPLAZSdrOA
40pakiEBtVs
vQ9ubZzX78s
nHvGhclktz0
lv8jQbW5v0c
ENUrM63tHhY
6Cn4D2W9ZYI
w0JNPFqgEMM
DZeRkq4viEU
gKgCSFZALkE
qsyFbyX5PuY
ZE-GjQy-PLc
dhfLBTPcG6U
AU_2AT2Kk10
TBFpiv3uNA4
S9BCgJNwJKM
oO7j-JKpjik
Bq1R0QuBNfI
IDFL1eCueEk
xHyB

N1aXgLtMXNA
pmanhxFZLX4
DyG5c6JP8c4
ZvIV-HIBQCQ
5djf6ve18g4
WO-oNYMF8BM
QNf7jj3oyNs
mOO_pMX8D0c
3I0loKd5LrE
bllEtTo5QKc
l5xqg4N8-KI
FnoiaRk7eqM
AMSDpmkBYI4
1KLk0uIbKKU
788pU7FsDBA
VjaLSrrlQGI
a-Vs0Br7HgQ
L-nNKidcGCo
Y2m0FPjCFEI
Zp9QvhheHyE
bPUwqHwqLkw
zInlom7xvUM
tKGc6zPTduI
25TAqOSpzlU
OIhllNHpk3M
sh3trs8s8kI
iLT6IDXggKE
x7plkR4S8Ec
b0EoyTzcFOI
93LxXFnEFZY
25VBk8AOqRQ
3ezoCvDZDkU
nznsdcazwqc
syVs_nUCGfU
ceWjdTMY_YQ
CAJ6IE-lKP4
wskG18saKk0
NqYnSbyuqBI
vuxyluwrO0k
3aZzl97e5S8
cOe8crSLWSw
3Ymwbhc3Qec
tk73Z0Jgu8I
23-CZjT-81w
IYDQfAaXHKc
UG0Y128PbTI
j7xGw3CBySU
N9YDe4H-OC8
TfZ6vCOzdTE
udCwBscdtKQ
slaudsOaPbs
chXFyNUZN6E
gftP0XHbb1M
3-ECJOBJmkg
JW3KE8pSKis
4kfudyahJcc
oxmnQmo41mk
IkfOGl_ZK-I
Ky9tqtrtDTY
sn_DFZJCc7U
deW8SdB0DzI
arj7oStGLkU
eIho2S0ZahI
6Af6b_wyiwI
qDiGIVy4eK4
735bPr1DK-4
xeBiUDbTOkQ
nJHzC7sWxFs
ZxPwZ2MN6O4
sKZiYoOqnBs
0LmtIWvo0ZQ
XpD239tqMI0
0ayPcQwXul4
JiJodvco5GU
670ykvA4FCg
2bGOopmLtk4
HLwfOld-m7w
vJT5YTdPvRA
hKGYsVSH0lA
nFZ72C7Ax64
SYWOKNZYqQU
LwSz--ScoqA
SxjNkV4TBPM
L3ih

ODl-DYTyNyM
fwZ6JNfXezg
b1fa-Krw48Y
lkBxm42Cydk
kMuu-etCE70
BlGQjueh9Kw
pSudEWBAYRE
KSH-FVVtTf0
J_CFBjAyPWE
rgk158rLFHk
BKInZGOqcZ4
dnjNhcMsT1c
-7I7vF5SjmM
BFetTcrVWII
V0BYaTMbDl0
4Yy0qTqRTrg
m5b5xpVBwDg
SiQ86A5fO3w
BsEs-CLBbaU
yQJ6dQsEqj0
qwFLOBrADBs
3SxyQcxZbpA
-6RC2_QxBgw
42o2NI3YGpo
g48U_SafMsg
d6CsWVkXozM
kd1_iNWIo9c
q6xMHUt86SY
IuJpD4rsYjY
gCqg0DAefvs
oHTL-HfCCV0
Ixxa8v5vOZQ
eW5COQqnutQ
PHWaHgdx-P0
9OVYwnqvSUM
zObOciwQ804
2PkpC_jTVeg
pHcBAJrC_qM
HTGqmNt66UE
GjigplLG5Lg
wFwN8sr0gno
q577GGz5nRk
pFUKeD3FJm8
ba-CB6wVuvQ
CF3lFPW4E1o
SknpAR8EPHE
8Jed_au6LZQ
anwc-PTBFSA
feppHZYMXN4
yodd15leLjw
9GM6puGq-o8
zS7v58Y3ygQ
qN7uZjmgPQ8
VSyktsf-vaY
7PFeO9gbGBA
CeQY8IrKH-8
bOn3U3XzX1Q
jDWIRNCw2N8
tVEkk8Zqx1E
7J5OqWlMtZs
50ns0bilP0I
5uV8Hra09JM
Zsu0RGeJkQg
ieEChe5MV-Y
1AIPEQgurls
LmYWMB-u73M
M9WtLebogS4
vsKNyWZ6ZEs
wHHvQdKnugg
Q42BgGVAO0Q
_6B4UK4tNSc
MmkQ6vMz1e4
VJ-vAUkktTE
uBCQi-E6e6s
jV8NQoLxn3w
MfIvdJmPoAk
mH9lOXR2HnY
USTWdVERhkQ
A2R_w_f7FGE
C23-l1yEGuI
-g3lVez6m_U
999xYufxbKM
aGSHLEd4S3Y
FA5a

...

fwyIuJFdKiQ
ITQoltKKW2E
8psO1zBHqJ0
8I8husXcziY
fTpgc4lH3RI
z8EjSFKsodM
kuRn2S7iPNU
NWF2JBb1bvM
bq5SGSCZe4E
BuW8BYKv688
D_8WShgZLQM
eIQ20RvhUhg
YAxB3Or2-Sg
QnSn7HLk_-0
x_IC5recCuw
AR7jBzqhGL0
dUWzMAHb_fg
cWX_iTwRUto
BBOvCZkOIiQ
Qq7YolZIYvw
_usJoVhVJtA
NYyA9Ks3TKk
Bz0qasPwuZk
4hULGasjFdY
-rPc5fvXf7Q
09rJUqShg-Y
JHhIVFVRCls
TlniuyuiB4U
Iv0cheeWwxw
7Qdwp2KRE0w
c6nf0ursWZs
DMqQtylat3U
a7oOnLF-s-Q
q9zI9-ss1qs
oToT5ACaQnA
-V6eefi9jQI
qUVZAfHg7jU
KqTjt0JGLcc
-J7-4G4m82g
ef-pvuP2IUc
4CDvcGlTcKA
03TUdXU_vYw
nNTPQUKowjA
QQo1qRyQLns
uNCWX6433EU
W5KWD9e0msU
lzq78MIkT2s
MOItoJFhAZY
kXX9M2zEyV4
TtUxFO1BkHQ
_ZbXoWL6hGc
ony2sdr_jjc
jBPMh_6ZY_8
qXdC3O-AWvg
T3retnQVSOs
gvru_VW_IiE
JMNUF1DFlaw
sqelF9LaAlQ
0TRokMB9AnI
2K_GE4dMRrM
RgNJkboVBDU
jGfme6RjThg
iO9PMgVdUYY
8sxX--U4c8w
3pSW8-3Zth0
yKuLpc4GP1o
g3r2b8lulBs
8FGLSlgeZYU
4hy05EOvIGQ
aGpOkJWOrfc
SYom2fimgyI
RqHC4UbW2MY
dg1iF90sFv4
u68ygf7Lb7Y
y5Hqp-T9euo
Gn_eMUFlLu4
PJMoOizRZqM
l55mgL_vcV0
PBD8-2Yki90
4JRglU83uAY
roYNh3ni5EE
2HY6jJkmGf0
Nj_FieStvOM
M6Yc

0BycXC7jWJk
SNav76IuILo
7aL3l7TGkBs
tQ4Oj3hCod4
QuQ-HHjmvOY
d7mpCS8rlgM
EXmA3X8O-uY
JilCqDzyy8s
DxX3-ZX5P1k
dC_vE4_a8mI
b2dfnWZ2a34
Pt6Pc1ngweE
YsztlQcTTS4
Pl-gBMQfGLM
XtAzUSoRRiM
9aXSAMei5MY
L8bmU6emFSE
eNXaznR8O7s
27IKXhn93kM
pPnkXBorxMY
2EK2r165A-g
iJygO8yyc0Y
mUhfScvRzBA
8Pga3brliIw
EVxd-UgbYzU
vPVsaG1mqWE
c8Smonl_Iio
s4wg9G35FRo
qhia8sf2MLU
6YRNNYFcvkU
27LS3cQRqIo
T0NINJOuaf4
pqabxBKzZ6M
1cB5JZcosr4
rjYoBqh1zVI
Pcq7BSYl8Zk
vti-qgpdy1M
FGetizODxpc
g8kObpRT9Hs
tJILWlhTm44
_FepEDId6ek
hFWB79r-l0A
czuPk7cC7UM
n_KnG1whj8s
bIbnoLdBxso
upl6A8LQLag
IUkbcSyinIo
jdU_X3Q9UFk
oqNU0-ORkbA
ZEM4eyMjk7Q
iUb-dF3AJes
xRtjeWsIscc
nccpEWtfGsU
eh0Z2pmzHm8
Cq9LytJ8NTs
gOrvKPDsfIA
xO61iLiRKeQ
hWt10PEoHC4
YOBMIBD5des
NVoFzM4cKF4
6Y5ZRaiVdKc
L54f8Ok1Z5w
wNCD3rLbCBk
3thqaEec64g
NNFxPqFthB4
5AtZeIjeu98
FSZVbdnApLY
DAu7bDtuVUI
kdTR67GLeCc
ULUzhrJ7zGk
bLwZiriLHaI
mefPkbiABt0
P0gbjSvPVwA
ZTY4vPyeJ-Y
4fJBY7O2Wjs
Dg1g62s3WJE
HIonKbKM-tE
NaMtktnlb4A
S20OLMrEmz8
fwo3EQBsTeg
egh_wWCEVyM
MBlFJ9AIHEg
LcEpple5Ino
Wk7U

WvpO8To1qFg
th_LYe97ZVc
Y-IoluTnuKs
xZjTTLRESRg
hIP1Ip2HgKE
hhJkQ9S4SJE
mAZipHo1dYE
lCoA-9DLY0A
fbde2ZETzEw
i3Z_33nakdA
-uzBl9vgpG0
d1ijSlrNHPk
8u2yIAD1ZHY
JOwIBblhYG4
Vich7u_AmOA
s4SBIBW_s-I
jIfUE4UJSrM
10KZS8zyaHc
omK_8TIGzFw
4Uzo8km08_c
yZx9JAs1-YI
1vP08XEr_BM
8vLI4PBRsl4
ZNK7cS2tJgg
OML9AVR10PQ
xWKPvwh2Wc0
rkWfxOp6m9w
ihvynXm77C8
CHSEF5GqC5k
2puBmXfi9Z0
WxgA_b1tsa0
zliaeMdpy4M
wGz3SJe5d1w
dIiSrcMzdEc
QlwNLIJJEzg
ZXHaEeL_i7A
v_sP-X-lBFU
UUN-cOvAkos
J61eCE-dKZM
7kd2GT6r_0E
4bUGq48H5_A
8WWv5_k8D14
S-ezhTXPVGU
Q-GOFPM01d0
blHVYY0dr3U
di2Kx447dhE
y0T8JCReh1w
LSV__p6E7wk
F6sjRs5nUCo
kfmR-1ArAok
rtJoBt82FgM
_qsX8N0KJRg
skrwjSRR9IA
rhD2SabYvu0
vkQRKe9oi3w
6xj1HVFdqWc
BNpQlvvtEgI
b8dvhBPrGtw
pU_MIfciKFE
pY9zDRc377Q
F-OgKyUuFfI
zX0AV6yxyrQ
Ylx4gWWYeyE
hJL8b1pwHdw
NiIW-6gYIZ0
mAxvK2aGtQU
d8kCTPPwfpM
ad_higXixRA
vzfhvO-2MP4
qeBJB9z2PMQ
42R9VyE2v9A
3IPV4stOaQI
Svz2jqHQ3f0
SjhcbqbrXHA
xvoT3atF1o4
P_l326Kz5zo
3csBtwl7Irc
3scp7oR0abU
faJWRP_qwqk
9I2JQ-P03HE
hJe4By2yurA
vUwBHR767qs
6qfYzYt41uE
TNMa

4lWrNhbwCe8
zRaJuSQI_sg
z0KKHel88Vo
gXdYqG5B6vY
elWE2jqjwVc
2iC9eOCxYbA
HsZrlCdOnuQ
7dsT649SYCM
U8puQBPvdq4
SqDt_UKmYfc
3262w5HK1NE
zGZr3TpGndg
z-IR48Mb3W0
Uj3_KqkI9Zo
Y6e_m9iq-4Q
nZC-OnTs1Z8
KxSAP3xZleU
N3fTeZQF9MY
JrHDeSMvnt4
kcmWknKtJkk
EoojNzIL2cQ
6Qu2OSpZglo
cAFOppCSUHI
i_iY-Qyzswk
K3KoT_HdphI
y7Nzhm_PuAo
AFEJwdlhOmE
0UcRAE94yXU
ruw1elYIQo4
WK5XYG-dH-k
FJBgzX2HMe8
2nYozPLpJRE
OGWPDHrUWko
OCbc3eEpHf4
tOKBgfHoSes
Fx6TSLUjMuA
3KA_YWDZwM8
PKoIyuodyA4
2YZi1VTOxY4
rA9CbdJSnOU
y8VNGZzzYjQ
02uHs0KGLLo
-veqti6OuUs
7k44Mle7bFM
pV-WEd4-FoQ
TmMD04F-y9A
oB4LfBnaGDA
d9B6m9QtiGY
QWn3oE2HDnc
1uQDDQlMpPU
TGIHp4divB0
Qwst8DFBEJk
vR-64Kpe0Kw
HS1GksfhPLQ
1RxYGn2cbZE
G84Pv7-yQJg
beuC1ukWc5I
fc0e3N7Xw3s
wdmGEdNIsBA
9xwh6VVVcVg
4MQ0j0gWjkc
Pc5fDI6m5X4
dPSy3QDq0-w
y5CJhgYf-Hc
RX4k1BQFS3c
amkaARuFPus
qTmHuavOXNg
u37lG0BMFZk
9rkFbKVownA
3YTJrwn3kBI
Am45Vld1tBk
I2jHGzFU2hk
OTkCh02V74o
lE1oZBXUqgE
TrBbLlXP3Ms
MD8ea6rkNaM
LLD9lC64_h4
YH-xnsnl3Rg
jktrATqDojo
4i9_rANO48U
ct6KQ841cO8
5T3hQTGxfK0
onjxqpKUQ34
Qwoo

XwmOf9NBatM
0cF0ovA3FtY
SqpVHk2O778
0VQEJARHJ44
eUUbIF8lOoc
aa3gTkJb8iQ
ohSbhKQ-VWo
_uDK7AC7sv0
aUkTTci5Hak
lvTL_lnN5-Q
LTXfx4h4iPs
kVdsmbtKuKE
r8IfXPm9zrU
3TRl4edM_74
ad1oLRgWDHQ
bGCKN2lvuuQ
DMlEwbXmLx8
YfOYI0AXFvU
AUps1qgnoUU
C-DXAaSiGxA
dJmlDPamgso
fiQeuDi98hY
NAgkl2AOiLs
OfLcG4oNPdQ
uJ4opRiXe5o
tajfY7YpKa0
w_yws7db2hY
oMlxxAaoWHY
xExhtqat63E
da5x-__kvGY
8wc-Wm28ras
21CWrexxpi8
lwkGtioC_20
jPQ87J_5qyw
0SPwwpruGIA
1VZdAU2oIrY
bMzw3IsuJPg
ZC2qM5cnaZE
sf1Eva_-eTs
JZvr_X4S0BM
B9Usox2Doos
66EraYlrOQQ
j4UbDZG6N1M
0yxmi0G_p_g
57ev0uPhutQ
tGtxFVPYm_Q
N7TqjnPbWNA
Cc3AOtlyx3Q
a5V6gdu5ih8
_UbDeqPdUek
0fWtpVDSh9w
Y7OYaDia0BM
c7Q1j8yftJk
gUiY8uETcrQ
tO0WuF_SC9A
3oyGDXr3qMM
JfJyh8Ce4as
Opba8K_5cY4
1BWjneP_DUY
z0uPjIz7sFw
ctrePeyir3I
rXf4bgU3GRA
BXkgc9I-yWY
M0vpn6YVwiI
mUY2nWqdPxA
Y1IUFGh2T_E
IrJLeWh3e8k
seTzJN-ugG0
C1v8wbI1wD8
CjZn8k2zqNQ
rk7NYXql2dc
BJoH9XVlrsE
GVP2Q7NOzTc
iaIqc_EJ9_M
PQ8ihL22rno
M0uO8X3_tEA
xqVBoyKXbsA
tgm_QjUiwlk
UT11qDFDQOU
Qiw-sLUzsPE
fYivEj35uW4
tkaQUofuSmg
Db6OJZTrw-s
Vv01

In [22]:
with open('Data/video_metadata_from_channels.jsonl','w',encoding='utf-8') as file:
    for line in video_metadata:
        json.dump(line,file,ensure_ascii=False)
        file.write('\n')

In [ ]:
#get comments from videos from channels (keep at most 10**5 comments for each video due to time constraints)
for video in videos_to_collect['videoId']:
    if video in video_collected:
        continue
    try:
        comments=collector.getComments(video)
        while comments=='fail':
            print('failed once')
            comments=collector.getComments(video,limit=10**5)
        video_collected.add(video)
    except Exception as e:
        if 'disabled comments' in str(e):
            print(f'Video {video} has disabled comments')
            video_collected.add(video)
            continue
        else:
            print(e)
    print(f'Collected {len(comments)} comments for {video}')
    with open('Data/comments_from_channels.jsonl','a',encoding='utf-8') as file:
        for comment in comments:
            json.dump(comment,file,ensure_ascii=False)
            file.write('\n')